In [6]:
import numpy as np
import pandas as pd
#from sklearn import datasets, linear_model
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

print("Matplotlib version", matplotlib.__version__)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Matplotlib version 3.1.1


In [7]:
%%time
# From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

Wall time: 0 ns


In [8]:
#os.chdir(r"C:\Users\박유경\Desktop\project")
jeju = pd.read_csv('jeju_financial_life_data.csv', header='infer', na_values=[' '])
jeju= reduce_mem_usage(jeju)
jeju.head()

Memory usage of dataframe is 2.15 MB
Memory usage after optimization is: 0.94 MB
Decreased by 56.0%


,zip_cd,year_month,x_axis,y_axis,sex,age,job_majorc,job_smallc,job_public,job_profession,...,avg_debt,avg_debt_credit,avg_debt_noneb,avg_debt_mortgage,avg_debt_deposit,avg_debt_collateral,avg_credit_rat,medium_resid_rat,large_resid_rat,vehicle_own_rat
0,63299,201902,126.5625,33.50,1,54,0.0,0.166626,0.000000,0.000000,...,24285196,11419148,8863595,55550000,0,55550000,5,-999999.0,-999999.0,0.000000
1,63560,201902,126.5000,33.25,1,59,0.0,0.125000,0.000000,0.000000,...,108571077,25896955,13317443,0,200000000,74421721,4,-999999.0,-999999.0,0.000000
2,63560,201902,126.5000,33.25,2,49,0.0,0.250000,0.000000,0.000000,...,26424335,12015311,12015311,40833359,0,40833359,2,-999999.0,-999999.0,0.000000
3,63560,201902,126.5000,33.25,1,54,0.0,0.181763,0.090881,0.090881,...,647716901,39952556,21558153,36697792,0,706300496,4,-999999.0,-999999.0,0.166626
4,63560,201902,126.5000,33.25,1,49,0.0,0.250000,0.000000,0.000000,...,248232725,35712501,5568751,0,0,626810674,5,-999999.0,-999999.0,0.000000


In [38]:
#jeju['age_young']=np.where(jeju['age']<40 ,1,0)
jeju['monthly_income']=(jeju['avg_income']/12)
jeju['monthly_spend']=(jeju['avg_spend']/3)
jeju['spend_income']=(jeju.monthly_spend/(jeju.monthly_income+1))
#jeju['age_young']=jeju[(jeju.age >=20) & (jeju.age <=35)]
#jeju['spend_income'].max()
jeju


#소득대비 소비가 안정성 지표 얘가 1보다 커야해

,zip_cd,year_month,x_axis,y_axis,sex,age,job_majorc,job_smallc,job_public,job_profession,...,avg_debt_collateral,avg_credit_rat,medium_resid_rat,large_resid_rat,vehicle_own_rat,credit_hl,monthly_income,monthly_spend,spend_income,age_young
0,63299,201902,126.5625,33.50000,1,54,0.000000,0.166626,0.000000,0.000000,...,55550000,5,-999999.000000,-999999.0,0.000000,0,3.099000e+06,8.705397e+05,0.280910,0
1,63560,201902,126.5000,33.25000,1,59,0.000000,0.125000,0.000000,0.000000,...,74421721,4,-999999.000000,-999999.0,0.000000,1,2.948167e+06,6.625753e+05,0.224741,0
2,63560,201902,126.5000,33.25000,2,49,0.000000,0.250000,0.000000,0.000000,...,40833359,2,-999999.000000,-999999.0,0.000000,1,2.676583e+06,1.176556e+06,0.439574,0
3,63560,201902,126.5000,33.25000,1,54,0.000000,0.181763,0.090881,0.090881,...,706300496,4,-999999.000000,-999999.0,0.166626,1,5.395500e+06,1.622826e+06,0.300774,0
4,63560,201902,126.5000,33.25000,1,49,0.000000,0.250000,0.000000,0.000000,...,626810674,5,-999999.000000,-999999.0,0.000000,0,2.464833e+06,1.736563e+06,0.704535,0
5,63560,201902,126.5000,33.25000,2,39,0.000000,0.125000,0.000000,0.125000,...,0,2,-999999.000000,-999999.0,0.000000,1,2.925833e+06,1.463792e+06,0.500299,1
6,63560,201902,126.5000,33.25000,1,64,0.000000,0.090881,0.000000,0.000000,...,1272316958,4,-999999.000000,-999999.0,0.000000,1,4.353917e+06,6.909920e+05,0.158706,0
7,63560,201902,126.5000,33.25000,1,44,0.000000,0.300049,0.099976,0.000000,...,278042709,3,-999999.000000,-999999.0,0.181763,1,5.235167e+06,1.175988e+07,2.246324,0
8,63539,201902,126.4375,33.25000,1,54,0.090881,0.181763,0.000000,0.000000,...,157776509,4,-999999.000000,-999999.0,0.090881,1,3.574500e+06,1.292240e+06,0.361516,0
9,63539,201902,126.4375,33.25000,1,44,0.000000,0.125000,0.000000,0.000000,...,388694877,5,-999999.000000,-999999.0,0.000000,0,2.884250e+06,1.700299e+06,0.589511,0


In [28]:
# x=jeju[['spend_income','avg_debt']]
# y=jeju[['age_young']]

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
# #https://no17.tistory.com/202


In [29]:
# from sklearn.linear_model import LogisticRegression
# log = LogisticRegression() #로지스틱 회귀분석 시행
# log.fit(x_train, y_train) #모델의 정확도 확인

# #print(x_train)
# print('학습용 데이터셋 정확도 : %.2f' % log.score(x_train, y_train))
# print('검증용 데이터셋 정확도 : %.2f' % log.score(x_test, y_test))

학습용 데이터셋 정확도 : 0.10
검증용 데이터셋 정확도 : 0.08


In [30]:
# from sklearn.metrics import classification_report
# y_pred=log.predict(x_test)
# print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          24       0.38      0.10      0.16       250
          29       0.00      0.00      0.00       230
          34       0.00      0.00      0.00       239
          39       0.00      0.00      0.00       282
          44       0.00      0.00      0.00       268
          49       0.00      0.00      0.00       263
          54       0.00      0.00      0.00       289
          59       0.08      1.00      0.15       239
          64       0.00      0.00      0.00       257
          69       0.00      0.00      0.00       218
          74       0.00      0.00      0.00       216
          79       0.25      0.01      0.01       186
          99       0.00      0.00      0.00       190

    accuracy                           0.08      3127
   macro avg       0.05      0.09      0.02      3127
weighted avg       0.05      0.08      0.02      3127



In [32]:
# import statsmodels.api as sm

# x= np.asarray(x)
# k = np.asarray(jeju[['age_young']])



# logit=sm.Logit(k,x) #로지스틱 회귀분석 시행
# result=logit.fit()

Optimization terminated successfully.
         Current function value: 0.541715
         Iterations 3


In [33]:
# result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                10422
Model:                          Logit   Df Residuals:                    10420
Method:                           MLE   Df Model:                            1
Date:                Mon, 05 Aug 2019   Pseudo R-squ.:                  0.1285
Time:                        17:26:33   Log-Likelihood:                -5645.8
converged:                       True   LL-Null:                       -6478.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1         -2.352e-07   8.33e-07     -0.282      0.778   -1.87e-06     1.4e-06
x2         -1.335e-08   3.11e-10    -42.976      0.000    -1.4e-08   -1.27e-08
==============================================================================
"""

In [51]:
#jeju_young=jeju[(jeju.age >=20) & (jeju.age <=39)]
jeju_young=jeju[(jeju.age >=20) & (jeju.age <=35)]
jeju_young

,zip_cd,year_month,x_axis,y_axis,sex,age,job_majorc,job_smallc,job_public,job_profession,...,avg_debt_collateral,avg_credit_rat,medium_resid_rat,large_resid_rat,vehicle_own_rat,credit_hl,monthly_income,monthly_spend,spend_income,age_young
11,63539,201902,126.4375,33.25000,1,29,0.000000,0.428467,0.000000,0.000000,...,0,5,-999999.000000,-999999.000000,0.000000,0,2.366417e+06,8.636920e+05,0.364979,1
13,63059,201902,126.4375,33.46875,1,34,0.000000,0.250000,0.000000,0.000000,...,0,5,-999999.000000,-999999.000000,0.000000,0,2.730583e+06,2.638807e+06,0.966389,1
14,63059,201902,126.4375,33.46875,2,34,0.000000,0.500000,0.000000,0.000000,...,83751643,4,-999999.000000,-999999.000000,0.000000,1,2.362667e+06,1.847553e+06,0.781977,1
26,63306,201902,126.5625,33.50000,1,29,0.000000,0.111084,0.000000,0.000000,...,132600000,4,-999999.000000,-999999.000000,0.000000,1,2.416667e+06,1.095126e+06,0.453155,1
30,63306,201902,126.5625,33.50000,1,24,0.000000,0.000000,0.000000,0.000000,...,0,4,-999999.000000,-999999.000000,0.000000,1,2.000000e+06,9.146533e+05,0.457326,1
38,63307,201902,126.5625,33.50000,2,24,0.000000,0.000000,0.000000,0.090881,...,0,4,-999999.000000,-999999.000000,0.000000,1,1.777750e+06,9.949540e+05,0.559670,1
39,63307,201902,126.5625,33.50000,2,34,0.125000,0.375000,0.000000,0.000000,...,70578063,5,-999999.000000,-999999.000000,0.250000,0,2.037917e+06,1.585213e+06,0.777859,1
46,63307,201902,126.5625,33.50000,1,24,0.000000,0.090881,0.000000,0.000000,...,0,4,-999999.000000,-999999.000000,0.000000,1,1.944417e+06,8.407410e+05,0.432387,1
49,63275,201902,126.5625,33.50000,1,29,0.071411,0.214233,0.071411,0.023804,...,94485116,4,1.000000,0.222222,0.045441,1,2.580417e+06,1.344437e+06,0.521015,1
51,63275,201902,126.5625,33.50000,1,24,0.000000,0.045441,0.000000,0.000000,...,0,4,1.000000,0.500000,0.000000,1,1.948750e+06,6.126990e+05,0.314406,1


In [52]:
# 선형회귀

import statsmodels.api as sm

reg=sm.OLS.from_formula("age ~ spend_income +avg_debt", jeju_young).fit()

In [53]:
reg.summary()
#유의하다 5%(0.05, 0.01 둘다 유의)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    age   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     120.6
Date:                Mon, 05 Aug 2019   Prob (F-statistic):           1.24e-50
Time:                        17:42:06   Log-Likelihood:                -6734.4
No. Observations:                2420   AIC:                         1.347e+04
Df Residuals:                    2417   BIC:                         1.349e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       28.1201      0.096    291.611      0.000      27.931      28.309
spend_income -6.682e-06   2.41e-06     -2.767      0.006   -1.14e-05   -1.95e-06
avg_debt      2.403e-08   1.58e-09     15.186      0.000    2.09e-08    2.71e-08
==============================================================================
Omnibus:                       17.988   Durbin-Watson:                   2.138
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               16.232
Skew:                          -0.154   Prob(JB):                     0.000299
Kurtosis:                       2.742   Cond. No.                     7.38e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.38e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [55]:
#피어슨 상관계수
corr = jeju_young.corr(method ='pearson')
corr

,zip_cd,year_month,x_axis,y_axis,sex,age,job_majorc,job_smallc,job_public,job_profession,...,avg_debt_collateral,avg_credit_rat,medium_resid_rat,large_resid_rat,vehicle_own_rat,credit_hl,monthly_income,monthly_spend,spend_income,age_young
zip_cd,1.000000,NaN,0.453536,-0.660334,-0.003928,-0.001207,0.061221,-0.007352,-0.018617,-0.017993,...,-0.024192,0.064371,-0.087189,-0.087189,0.084584,-0.077145,-0.022319,-0.053200,0.013615,NaN
year_month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_axis,0.453536,NaN,1.000000,0.209522,0.009231,0.011285,-0.002112,-0.020026,0.021345,0.015606,...,-0.015215,0.004460,0.002908,0.002908,0.022374,-0.005450,-0.003741,-0.004522,-0.013422,NaN
y_axis,-0.660334,NaN,0.209522,1.000000,0.006599,0.007641,-0.072002,0.017604,0.028694,0.033149,...,-0.005173,-0.064434,0.141548,0.141548,-0.068234,0.058429,0.011828,0.042965,-0.025426,NaN
sex,-0.003928,NaN,0.009231,0.006599,1.000000,0.002052,-0.065743,-0.092628,-0.331393,0.338952,...,-0.098550,-0.343381,0.020445,0.020445,-0.233956,0.256996,-0.319150,0.084685,-0.002501,NaN
age,-0.001207,NaN,0.011285,0.007641,0.002052,1.000000,0.404860,0.594511,0.385520,0.218390,...,0.237489,-0.221000,0.013305,0.013305,0.455036,0.013694,0.787258,0.767815,-0.062782,NaN
job_majorc,0.061221,NaN,-0.002112,-0.072002,-0.065743,0.404860,1.000000,0.203826,0.108510,0.044921,...,0.089615,-0.071414,-0.015536,-0.015536,0.244367,-0.001996,0.399867,0.330332,-0.027154,NaN
job_smallc,-0.007352,NaN,-0.020026,0.017604,-0.092628,0.594511,0.203826,1.000000,0.220891,0.026993,...,0.185524,0.007305,-0.003458,-0.003458,0.315282,-0.114235,0.506195,0.496440,-0.072779,NaN
job_public,-0.018617,NaN,0.021345,0.028694,-0.331393,0.385520,0.108510,0.220891,1.000000,-0.011955,...,0.140851,-0.037165,0.029093,0.029093,0.260249,-0.021896,0.457561,0.327106,-0.040548,NaN
job_profession,-0.017993,NaN,0.015606,0.033149,0.338952,0.218390,0.044921,0.026993,-0.011955,1.000000,...,0.028579,-0.211088,0.021956,0.021956,0.000663,0.106080,0.082190,0.230439,-0.036033,NaN


In [62]:
#소득대비 소비가 안정성 지표 얘가 1보다 커야해
#jeju['spend_income']=(jeju.monthly_spend/(jeju.monthly_income+1))
jeju_young['household_income_index']=np.where(jeju_young['spend_income']>1,1,0)
jeju_young

,zip_cd,year_month,x_axis,y_axis,sex,age,job_majorc,job_smallc,job_public,job_profession,...,avg_credit_rat,medium_resid_rat,large_resid_rat,vehicle_own_rat,credit_hl,monthly_income,monthly_spend,spend_income,age_young,household_income_index
11,63539,201902,126.4375,33.25000,1,29,0.000000,0.428467,0.000000,0.000000,...,5,-999999.000000,-999999.000000,0.000000,0,2.366417e+06,8.636920e+05,0.364979,1,0
13,63059,201902,126.4375,33.46875,1,34,0.000000,0.250000,0.000000,0.000000,...,5,-999999.000000,-999999.000000,0.000000,0,2.730583e+06,2.638807e+06,0.966389,1,0
14,63059,201902,126.4375,33.46875,2,34,0.000000,0.500000,0.000000,0.000000,...,4,-999999.000000,-999999.000000,0.000000,1,2.362667e+06,1.847553e+06,0.781977,1,0
26,63306,201902,126.5625,33.50000,1,29,0.000000,0.111084,0.000000,0.000000,...,4,-999999.000000,-999999.000000,0.000000,1,2.416667e+06,1.095126e+06,0.453155,1,0
30,63306,201902,126.5625,33.50000,1,24,0.000000,0.000000,0.000000,0.000000,...,4,-999999.000000,-999999.000000,0.000000,1,2.000000e+06,9.146533e+05,0.457326,1,0
38,63307,201902,126.5625,33.50000,2,24,0.000000,0.000000,0.000000,0.090881,...,4,-999999.000000,-999999.000000,0.000000,1,1.777750e+06,9.949540e+05,0.559670,1,0
39,63307,201902,126.5625,33.50000,2,34,0.125000,0.375000,0.000000,0.000000,...,5,-999999.000000,-999999.000000,0.250000,0,2.037917e+06,1.585213e+06,0.777859,1,0
46,63307,201902,126.5625,33.50000,1,24,0.000000,0.090881,0.000000,0.000000,...,4,-999999.000000,-999999.000000,0.000000,1,1.944417e+06,8.407410e+05,0.432387,1,0
49,63275,201902,126.5625,33.50000,1,29,0.071411,0.214233,0.071411,0.023804,...,4,1.000000,0.222222,0.045441,1,2.580417e+06,1.344437e+06,0.521015,1,0
51,63275,201902,126.5625,33.50000,1,24,0.000000,0.045441,0.000000,0.000000,...,4,1.000000,0.500000,0.000000,1,1.948750e+06,6.126990e+05,0.314406,1,0
